In [1]:
import scipy.io
import numpy as np
import pandas as pd
import gzip
import os

def get_pseudobulk_from_mtx_gz(mtx_gz_path, features_tsv_gz_path=None):
    """
    Compute pseudobulk counts from a Cell Ranger mtx.gz file.
    - Sums counts across all cells for each gene.
    - Optionally loads gene names from features.tsv.gz.
    
    Args:
    - mtx_gz_path (str): Path to matrix.mtx.gz
    - features_tsv_gz_path (str, optional): Path to features.tsv.gz for gene names
    
    Returns:
    - pd.DataFrame: DataFrame with gene indices/names and pseudobulk counts
    """
    # Read the mtx.gz file
    with gzip.open(mtx_gz_path, 'rt') as f:
        mtx = scipy.io.mmread(f)
    
    # Sum across cells (axis=1) to get pseudobulk counts per gene
    pseudobulk_counts = np.array(mtx.sum(axis=1)).flatten()
    
    # Create DataFrame with gene indices by default
    pseudobulk_df = pd.DataFrame({
        'gene_index': range(1, len(pseudobulk_counts) + 1),
        'pseudobulk_count': pseudobulk_counts
    })
    
    # If features.tsv.gz is provided, load gene names/IDs
    if features_tsv_gz_path and os.path.exists(features_tsv_gz_path):
        features = pd.read_csv(features_tsv_gz_path, sep='\t', header=None, compression='gzip')
        gene_ids = features[0].values  # First column: gene IDs
        gene_names = features[1].values  # Second column: gene names
        pseudobulk_df['gene_id'] = gene_ids[:len(pseudobulk_counts)]
        pseudobulk_df['gene_name'] = gene_names[:len(pseudobulk_counts)]
        # Reorder columns for clarity
        pseudobulk_df = pseudobulk_df[['gene_id', 'gene_name', 'pseudobulk_count', 'gene_index']]
    
    return pseudobulk_df
#Data from: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5687481
# Example usage:
# Assuming files are in 'outs/filtered_feature_bc_matrix/' as per Cell Ranger structure
mtx_path = 'GSM5687482_k562_rep2_matrix.mtx.gz'
features_path = 'GSM5687482_k562_rep2_features.tsv.gz'  # Optional

pseudobulk_df = get_pseudobulk_from_mtx_gz(mtx_path, features_path)
print(pseudobulk_df.head())  # Preview the result

           gene_id    gene_name  pseudobulk_count  gene_index
0  ENSG00000243485  MIR1302-2HG                 0           1
1  ENSG00000237613      FAM138A                 0           2
2  ENSG00000186092        OR4F5                10           3
3  ENSG00000238009   AL627309.1               155           4
4  ENSG00000239945   AL627309.3                24           5


In [2]:
# To save to CSV:
pseudobulk_df[['gene_name','pseudobulk_count']].to_csv('10X_pseudobulk_counts_rep2.txt', index=False,header=False, sep='\t')